In [23]:
import json
import math

import torch
import torch.utils.data as data
import numpy as np

from squad.squad_dataset import SquadDataset
from training.model_trainer import Trainer
from training.utils import *
from model.QANet import QANet

training_set = SquadDataset("./preprocessed_dataset/training_set_features.npz", False)
validation_set = SquadDataset("./preprocessed_dataset/validation_set_features.npz", False)
validation_set_eval = json.load(open("./preprocessed_dataset/validation_set_eval.json"))

In [15]:
training_set_loader = data.DataLoader(
    training_set, 
    batch_size=32,
    shuffle=False
)

validation_set_loader = data.DataLoader(
    validation_set, 
    batch_size=32,
    shuffle=False
)

In [4]:
word_emb = torch.from_numpy(np.load('./preprocessed_dataset/glove_embeddings.npz')['emb_mat']).double()
chr_emb = torch.from_numpy(np.load('./preprocessed_dataset/char_embeddings.npz')['emb_mat']).double()

In [5]:
config = json.load(open("./config.json"))
model = QANet(word_emb, chr_emb, config).double().to("cuda:0")

In [9]:
ema = EMA(model, 0.9999)
optimizer = torch.optim.Adam(params=model.parameters(), lr=1e-3, betas=(0.8, 0.999), eps=1e-8, weight_decay=3e-7)
cr = 1.0 / math.log(1000)
scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lambda ee: cr * math.log(ee + 1) if ee < 1000 else 1)

QANet(
  (input_embedding_layer): InputEmbeddingLayer(
    (word_embed): Embedding(83941, 300)
    (char_embed): CharacterEmbedding(
      (char_embeddings): Embedding(1309, 200)
      (conv1): Conv2d(200, 200, kernel_size=(1, 5), stride=(1, 1))
    )
    (highway): Highway(
      (t_gates): ModuleList(
        (0): Linear(in_features=500, out_features=500, bias=True)
        (1): Linear(in_features=500, out_features=500, bias=True)
      )
      (h_gates): ModuleList(
        (0): Linear(in_features=500, out_features=500, bias=True)
        (1): Linear(in_features=500, out_features=500, bias=True)
      )
    )
  )
  (embedding_encoder_layer): EncoderEmbeddingLayer(
    (conv1d): Reshape1Dconv(
      (conv1d): Conv1d(500, 128, kernel_size=(1,), stride=(1,))
    )
    (c_pos_encoder): PositionalEncoder(
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (q_pos_encoder): PositionalEncoder(
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder_blocks): ModuleList(
     

In [ ]:
model.train()
trainer = Trainer(model, optimizer, "cuda:0", scheduler, 5.0, ema, 1, "model_checkpoints")
trainer.train(10, training_set_loader, validation_set_loader, validation_set_eval)

100%|█| 82240/82240 [16:19<00:00, 83.98it/s, avg_loss=6.47, batch_ce=4.92, epoch=1, learning_rate=[0.001], time=F


Validating...


100%|█████████████████████████████████████| 5136/5136 [00:18<00:00, 272.79it/s, batch_ce=5.13, time=Feb-06_17-03]


{'EM': 21.904205607476637, 'F1': 32.5156667452638}
Saving checkpoint: model_checkpoints/epoch01_f1_32.51567_em_21.90421.pth.tar ...


100%|█| 82240/82240 [16:18<00:00, 84.02it/s, avg_loss=5.08, batch_ce=3.87, epoch=2, learning_rate=[0.001], time=F


...

100%|█| 82240/82240 [16:18<00:00, 84.01it/s, avg_loss=2.28, batch_ce=0.954, epoch=9, learning_rate=[0.001], time=


Validating...


100%|█████████████████████████████████████| 5136/5136 [00:18<00:00, 273.44it/s, batch_ce=2.41, time=Feb-06_19-16]


{'EM': 54.94548286604361, 'F1': 69.47861040226148}
Saving checkpoint: model_checkpoints/epoch09_f1_69.47861_em_54.94548.pth.tar ...


 93%|▉| 76512/82240 [15:10<01:08, 83.90it/s, avg_loss=2.18, batch_ce=0.9, epoch=10, learning_rate=[0.001], time=F